<a href="https://colab.research.google.com/github/Jatingpt/GenAI-Cold-Email-Generator-Project/blob/main/Gen_AI_Cold_Email_Generator_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ✅ Step 1: Install Required Packages
!pip install langchain langchain-community langchain-core langchain-groq chromadb pandas

# ✅ Step 2: Import Libraries
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import uuid
import chromadb

# ✅ Step 3: Initialize LLM (replace with your Groq key)
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama3-70b-8192"
)

# ✅ Step 4: Scrape Job Description
loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")  #for the future reference if the job is expire then use the other job reference link.
page_data = loader.load().pop().page_content

# ✅ Step 5: Prompt Template for Job Info Extraction
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# ✅ Step 6: Extract Job Info
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})

# ✅ Step 7: Parse Extracted Info
json_parser = JsonOutputParser()
job = json_parser.parse(res.content)

# ✅ Step 8: Load Your Portfolio CSV (Upload in Colab first)
df = pd.read_csv("/content/my_portfolio.csv")  # Must have columns: Techstack, Links

# ✅ Step 9: Create Vector DB Using ChromaDB
client = chromadb.PersistentClient(path='vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# ✅ Step 10: Add to ChromaDB (Run Once)
if collection.count() == 0:
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

# ✅ Step 11: Query Top Matching Portfolios (Fix: convert skills to list of string)
# ✅ Fix: Safely select first job if job is a list
if isinstance(job, list):
    job = job[0]  # pick the first job posting

skills_query = job['skills'] if isinstance(job['skills'], list) else [job['skills']]
results = collection.query(query_texts=skills_query, n_results=2)


# ✅ Flatten metadata list to extract links safely
metadatas = results.get('metadatas', [])
matched_links = [meta['links'] for sublist in metadatas for meta in sublist if isinstance(meta, dict) and 'links' in meta]

# ✅ Step 13: Create Prompt for Cold Email
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

# ✅ Step 14: Generate the Email
chain_email = prompt_email | llm
email_response = chain_email.invoke({
    "job_description": str(job),
    "link_list": matched_links
})

# ✅ Step 15: Print the Final Email
print("------ FINAL COLD EMAIL ------\n")
print(email_response.content)


------ FINAL COLD EMAIL ------

Subject: Enhance Retail Operations with AtliQ's Automation Expertise

Dear Hiring Manager,

I came across the job posting for an Athlète Nike (Sales Associate) at your Claye, Seine-et-Marne, France retail store. As a Business Development Executive at AtliQ, I understand the importance of streamlining retail operations to provide an exceptional customer experience.

At AtliQ, we specialize in developing tailored solutions to automate business processes, driving scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in AI and software consulting can help Nike enhance its retail operations, ensuring seamless integration of processes and improved customer satisfaction.

Our portfolio showcases our capabilities in developing innovative solutions. For instance, our work on Android TV portfolio demonstrates our ability to create engaging user experiences, while our Kotlin backend portfolio highlights our expertise in